In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#데이터 전처리

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense, Reshape, SeparableConv2D, Conv2D, BatchNormalization, Multiply, Layer,Attention, LayerNormalization, Add
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


In [ ]:
def custom_augmentation(image):
    image = random_erase(image, p=1, scale=(0.05, 0.05))
    return image

import numpy as np
import cv2

def random_erase(image, p=1, scale=(0.05, 0.05)):
    if np.random.rand() > p:
        return image

    h, w, _ = image.shape
    erase_area = np.random.uniform(scale[0], scale[1], 2)
    erase_h = int(h * erase_area[0])
    erase_w = int(w * erase_area[1])

    erase_x = np.random.randint(0, w - erase_w + 1)
    erase_y = np.random.randint(0, h - erase_h + 1)

    image[erase_y:erase_y + erase_h, erase_x:erase_x + erase_w, :] = np.random.randint(0, 256, (erase_h, erase_w, 3), dtype=np.uint8)
    return image

In [8]:
# 데이터 경로
train_data_dir = '/content/drive/MyDrive/Data/img_bylabel/train'
validation_data_dir = '/content/drive/MyDrive/Data/img_bylabel/val'
test_data_dir = '/content/drive/MyDrive/Data/img/test'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,  # 제안된 20도 회전
    width_shift_range=0.1,  # 제안된 20% 이동
    height_shift_range=0.1,  # 제안된 20% 이동
    zoom_range=(0.8, 1.0),  # 제안된 0.8~1 스케일
    horizontal_flip=True,  # 랜덤 좌우 반전
    fill_mode='nearest',
    brightness_range=[0.8, 1.2],  # 제안된 밝기 범위
    preprocessing_function=custom_augmentation  # 사용자 정의 함수 추가
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# 이미지 불러오기
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

NameError: name 'custom_augmentation' is not defined

#MobileNetV1 기반 모델링(실행환경 T4)

In [ ]:
# Base model 정의
def create_base_model(input_shape):
    # Load the base MobileNet model with pre-trained ImageNet weights
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False

    # Get the output from the fourth last layer
    layer_index = -14
    output_layer = base_model.layers[layer_index].output

    # Create the new model
    base_model = Model(inputs=base_model.input, outputs=output_layer)

    return base_model


# 패치 추출 레이어 정의
def patch_extraction_layer():
    return tf.keras.Sequential([
        SeparableConv2D(256, kernel_size=4, strides=4, padding='same', activation='relu'),
        SeparableConv2D(256, kernel_size=2, strides=2, padding='valid', activation='relu'),
        Conv2D(256, kernel_size=1, strides=1, padding='valid', activation='relu')
    ], name='patch_extraction')

# Pre-classification 레이어 정의
def create_pre_classification_layer():
    return tf.keras.Sequential([
        Dense(32, activation='relu'),
        BatchNormalization()
    ], name='pre_classification')

In [ ]:
def create_MobilePatchAttentionNet(input_shape, num_classes):
    base_model = create_base_model(input_shape)
    inputs = Input(shape=input_shape)
    # 기본 모델
    x = base_model(inputs)
    # 패치 추출 레이어
    x = patch_extraction_layer()(x)
    # GlobalAveragePooling2D 및 Dropout
    x = GlobalAveragePooling2D(name='gap')(x)
    x = Dropout(0.1)(x)
    # 사전 분류 레이어
    x = create_pre_classification_layer()(x)
    # Self-Attention 추가
    attn_output = Attention(use_scale=True)([x, x])
    x = Add()([x, attn_output])
    # 출력층
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model


# 모델 생성 및 컴파일
model = create_MobilePatchAttentionNet(input_shape=(224, 224, 3), num_classes=4)
model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()

17225924/17225924 [==============================] - 1s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 model (Functional)          (None, 14, 14, 512)          1627840   ['input_2[0][0]']             
                                                                                                  
 patch_extraction (Sequenti  (None, 2, 2, 256)            272128    ['model[0][0]']               
 al)                                                                                              
                                                                                                  
 gap (GlobalAveragePooling2

In [ ]:
# 학습률 조정 콜백
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=2, min_delta=0.005, min_lr=1e-7)

# EarlyStopping 콜백
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, min_delta=0.005, restore_best_weights=True)

In [ ]:
# 모델 훈련
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stopping]
)

Epoch 1/100
174/174 [==============================] - 3615s 21s/step - loss: 1.3696 - accuracy: 0.2974 - val_loss: 1.3778 - val_accuracy: 0.2878 - lr: 0.0010
Epoch 2/100
174/174 [==============================] - 417s 2s/step - loss: 1.1705 - accuracy: 0.4832 - val_loss: 1.3357 - val_accuracy: 0.3971 - lr: 0.0010
Epoch 3/100
174/174 [==============================] - 413s 2s/step - loss: 1.0831 - accuracy: 0.5348 - val_loss: 1.2312 - val_accuracy: 0.4208 - lr: 0.0010
Epoch 4/100
174/174 [==============================] - 413s 2s/step - loss: 1.0166 - accuracy: 0.5686 - val_loss: 1.4847 - val_accuracy: 0.4335 - lr: 0.0010
Epoch 5/100
174/174 [==============================] - 414s 2s/step - loss: 0.9793 - accuracy: 0.5911 - val_loss: 2.6739 - val_accuracy: 0.3242 - lr: 0.0010
Epoch 6/100
174/174 [==============================] - 410s 2s/step - loss: 0.9574 - accuracy: 0.6053 - val_loss: 1.4086 - val_accuracy: 0.4199 - lr: 0.0010
Epoch 7/100
174/174 [==============================] - 4

In [ ]:
# 파인튜닝 과정
base_model = model.layers[1]
base_model.trainable = True

num_freeze_layers = 43

# 하위 레이어 고정, 상위 레이어 해제
for layer in base_model.layers[:num_freeze_layers]:
    layer.trainable = False
for layer in base_model.layers[num_freeze_layers:]:
    if isinstance(layer, BatchNormalization):
        layer.trainable = False
    else:
        layer.trainable = True

# 모델 재컴파일
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 model (Functional)          (None, 14, 14, 512)          1627840   ['input_2[0][0]']             
                                                                                                  
 patch_extraction (Sequenti  (None, None, None, 256)      272128    ['model[0][0]']               
 al)                                                                                              
                                                                                                  
 gap (GlobalAveragePooling2  (None, 256)                  0         ['patch_extraction[0][0]

In [ ]:
# 모델 훈련
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stopping]
)

Epoch 1/100
174/174 [==============================] - 419s 2s/step - loss: 0.8898 - accuracy: 0.6269 - val_loss: 1.5133 - val_accuracy: 0.4918 - lr: 1.0000e-04
Epoch 2/100
174/174 [==============================] - 411s 2s/step - loss: 0.7912 - accuracy: 0.6781 - val_loss: 1.0510 - val_accuracy: 0.5984 - lr: 1.0000e-04
Epoch 3/100
174/174 [==============================] - 412s 2s/step - loss: 0.7126 - accuracy: 0.7164 - val_loss: 1.1421 - val_accuracy: 0.5738 - lr: 1.0000e-04
Epoch 4/100
174/174 [==============================] - 414s 2s/step - loss: 0.6684 - accuracy: 0.7391 - val_loss: 1.1149 - val_accuracy: 0.6311 - lr: 1.0000e-04
Epoch 5/100
174/174 [==============================] - 413s 2s/step - loss: 0.6450 - accuracy: 0.7524 - val_loss: 1.9208 - val_accuracy: 0.5455 - lr: 1.0000e-04
Epoch 6/100
174/174 [==============================] - 413s 2s/step - loss: 0.6083 - accuracy: 0.7704 - val_loss: 0.6990 - val_accuracy: 0.7168 - lr: 1.0000e-04
Epoch 7/100
174/174 [=============

In [ ]:
# 검증 데이터 확인
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

35/35 [==============================] - 60s 2s/step - loss: 0.6094 - accuracy: 0.7741
Validation Loss: 0.6094480752944946
Validation Accuracy: 0.7741348147392273


In [ ]:
# 모델 저장
model.save('/content/drive/MyDrive/MPANET-01_by.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
import pandas as pd

# 모델 파일 경로
model_path = '/content/drive/MyDrive/MPANET-01_by.h5'

# 모델 로드
model = load_model(model_path)

# 테스트 데이터 디렉토리 경로
test_data_dir = '/content/drive/MyDrive/Data/img/test'

# 테스트 데이터를 위한 ImageDataGenerator 생성
val_test_datagen = ImageDataGenerator(rescale=1./255)

# 테스트 데이터 생성기 생성
test_generator = val_test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# 테스트 데이터 전체 검증
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# 예측 값 얻기
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# 각 클래스별 정확도 계산 및 잘못 예측한 경우 정리
misclassified_summary = []

for i, label in enumerate(class_labels):
    indices = np.where(true_classes == i)[0]
    class_accuracy = accuracy_score(true_classes[indices], predicted_classes[indices])
    print(f"Accuracy for class {label}: {class_accuracy * 100:.2f}%")

    # 잘못 예측한 경우 저장
    misclassified_indices = indices[true_classes[indices] != predicted_classes[indices]]
    for index in misclassified_indices:
        true_label = class_labels[true_classes[index]]
        predicted_label = class_labels[predicted_classes[index]]
        misclassified_summary.append((true_label, predicted_label))

# DataFrame으로 정리
misclassified_df = pd.DataFrame(misclassified_summary, columns=["True Label", "Predicted Label"])

# 잘못 예측한 경우를 집계
misclassified_counts = misclassified_df.groupby(['True Label', 'Predicted Label']).size().reset_index(name='Count')

# 정리된 결과 출력
print("\nMisclassified Cases Summary:")
print(misclassified_counts)


Found 1137 images belonging to 4 classes.
36/36 [==============================] - 768s 22s/step - loss: 0.4032 - accuracy: 0.8399
Test Loss: 0.40316566824913025
Test Accuracy: 0.8399296402931213
36/36 [==============================] - 61s 2s/step
Accuracy for class anger: 65.36%
Accuracy for class happy: 96.98%
Accuracy for class panic: 94.18%
Accuracy for class sadness: 78.87%

Misclassified Cases Summary:
   True Label Predicted Label  Count
0       anger           happy      9
1       anger           panic     64
2       anger         sadness     24
3       happy           anger      6
4       happy           panic      1
5       happy         sadness      2
6       panic           anger     10
7       panic           happy      4
8       panic         sadness      2
9     sadness           anger     25
10    sadness           happy      4
11    sadness           panic     31


#Sub

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# 이미지 전처리 함수
def load_and_prepare_image(image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size)
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.0  # 정규화
    return img_tensor

# 감정을 예측하는 함수
def predict_emotion(model, img_path):
    print("이미지 로딩 중...")
    test_image = load_and_prepare_image(img_path)
    print("감정 예측 중...")
    prediction = model.predict(test_image)
    return prediction

# 모델 경로와 이미지 경로
model_path = '/content/drive/MyDrive/MobileNetV1-6.h5'  # 학습된 모델 경로
image_path = '/content/drive/MyDrive/testimg/5.jpg'  # 테스트할 이미지 경로

# 모델 로드
model = load_model(model_path)

# 예측 결과 얻기
predictions = predict_emotion(model, image_path)
emotion_index = np.argmax(predictions)
emotion_labels = ['화남', '행복', '슬픔', '충격']  # 실제 감정 레이블로 변경

print("예측된 감정:", emotion_labels[emotion_index])
print("각 감정의 비율:")
for i, label in enumerate(emotion_labels):
    print(f"{label}: {predictions[0][i] * 100:.2f}%")
